In [ ]:
import cv2
import imghdr
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data_dir = '/Users/giacomorossi/Desktop/progetti/Neural Network/Happy/foto'
images_exts = ['jpeg', 'jpg', 'bmp', 'png']

#preprocess of images on the fly
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in images_exts:
                print('image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            #os.remove(image_path)

In [ ]:
#creates batches
#class 1 -> sad
#class 0 -> happy
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

fig, ax = plt.subplots(ncols = 8, figsize = (20, 20))
for idx, img in enumerate(batch[0][:8]):
  ax[idx].imshow(img.astype(int))
  ax[idx].title.set_text(batch[1][idx])

In [ ]:
data_s = data.map(lambda x, y: (x/255, y))
scaled_iterator = data_s.as_numpy_iterator()
batch = scaled_iterator.next()

fig, ax = plt.subplots(ncols = 4, figsize = (20, 20))
for idx, img in enumerate(batch[0][:4]):
  ax[idx].imshow(img)
  ax[idx].title.set_text(batch[1][idx])

In [ ]:
len(data_s)*32 #batch size

train_size = int(len(data_s)*.7)
val_size = int(len(data_s)*.15)
test_size = int(len(data_s)*.15)
train = data_s.take(train_size)
val = data_s.skip(val_size)
test = data_s.skip(train_size*val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
model = Sequential()

#input layer corresponding to the shape of the images
#it has 16 filters to scan the images with the filter size of (3, 3) px.
#it as a stride of 1, it moves 1px per time
model.add(Conv2D(16, (3, 3), 1, activation = 'relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D()) #reduces the images looking at the max value of the size taken (2x2 by default -> halfs the images)

model.add(Conv2D(32, (3, 3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3, 3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten()) #brings all in one channel

model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid')) #output layer

print(model.summary())

model.compile(optimizer = 'adam', loss = tf.losses.BinaryCrossentropy(), metrics = ['accuracy'])
hist = model.fit(train, epochs = 20, validation_data = val)



In [ ]:
model.save('HappyImageClass')

In [ ]:
model = tf.keras.models.load_model('/Users/giacomorossi/Desktop/progetti/Neural Network/Happy/HappyImageClass')

In [ ]:
img = cv2.imread('INSERT IMAGE')
resize = tf.image.resize(img, (256, 256))

yhat = model.predict(np.expand_dims(resize/255, 0))
if yhat.round() == 1:
  print('sad person')
else: 
  print('happy person')

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()